# POF - Tributos diretos e transferências sob uma análise interseccional

In [ ]:
# Importando as bibliotecas necessárias para a manipulação dos dados
import pyreadr
import pandas as pd
import numpy as np

In [ ]:
import os
os.chdir('Insira/aqui/seu/diretório/de/trabalho')

## Introdução

Esse tutorial apresenta como produzir a base de dados utilizada na Nota de Política Econômica nº27 a partir das informações da Pesquisa de Orçamentos Familiares de 2017-2018. A metodologia empregada aqui segue os chamdos <a href='https://www.ibge.gov.br/estatisticas/sociais/saude/24786-pesquisa-de-orcamentos-familiares-2.html?=&t=microdados'>Tradutores de tabelas</a>, disponíveis na documentação da pesquisa, para identificação de rendimentos e se baseia em Gaiger,2020 para a classificação dos tributos. Os primeiros são compostos por duas categorias, *monetários* e *não monetários*, sendo os *não monetários* aqueles bens e serviços recebidos via provisão pública, doações privadas, trocas, produção própria e retirada do negócio.¹ O conjunto desses rendimentos compõem aquilo chamado pelo IBGE como *renda total*. A diferença na metodologia de consolidação entre esses rendimentos está no fato de que os *monetários* serão construídos para cada indivíduo e apenas depois repartidos entre os membros do núcleo familiar, chamado pelo IBGE por *unidade de consumo* (UC), nomenclatura que adotaremos daqui em diante; já os rendimentos *não monetários* serão agregados desde o início para toda a UC, sem a construção de informações desse tipo de rendimento para indivíduos. Os tributos também serão consolidados no nível da UC e depois per capitalizados.

1: Ver o Dicionário de variáveis também disponível na [documentação](https://www.ibge.gov.br/estatisticas/sociais/saude/24786-pesquisa-de-orcamentos-familiares-2.html?=&t=microdados).

A construção dessas variáveis utiliza de quase todos os questionários da POF. A descrição de quais elementos do orçamento familiar são mensurados por cada um dos questinários é feita no *Manual do agente de pesquisa*,² nele é possível identificar qual *quadro de produtos* está associado com qual questionário. A descrição de cada um dos produtos, seu quadro e código são encontrados no *Cadastro de Produtos*, também parte da documentação da pesquisa. Apenas como um exemplo da maneira de se navegar entre os vários documentos, vamos pegar o `IMPOSTO PREDIAL E TERRITORIAL URBANO (IPTU)` no *Cadastro de Produtos*. Ele é indicado como quadro `10` com código `1000601`. Como podemos ver no *Manual do agente de pesquisa* esse quadro pertence ao questionário *Despesa coletiva*, que considera alguns gastos comuns do núcleo familiar, e considera *Aluguel, impostos e outras taxas do domicílio principal no Período de Referência de 12 Meses*. Por fim, para que você entenda os ajustes que faremos adiante nos códigos dos produtos, repare que logo abaixo do código `1000601` no *Cadastro de Produtos* temos outro código associado apenas a `IPTU`. Devido a essas pequenas variações de nomenclatura, utilizaremos ao longo desse tutorial os cinco primeiros dígitos dos códigos, retirando a parte voltada para diferenciações como essa.

2:Também disponível na [documentação](https://www.ibge.gov.br/estatisticas/sociais/saude/24786-pesquisa-de-orcamentos-familiares-2.html?=&t=microdados) da pesquisa.

Feita essa primeira matrícula no material da pesquisa, vamos abrir nossas bases de dados. Todas elas estão disponíveis no [link](https://www.ibge.gov.br/estatisticas/sociais/saude/24786-pesquisa-de-orcamentos-familiares-2.html?=&t=microdados) dos *Dados*, enquanto o link *Programa de leitura* permite abrir os arquivos em formato `.fwf` (*fixed width format*) através do **R**. Repare que os arquivos `.rds`, que resultam do código fornecido pelo IBGE, devem estar no seu diretório de trabalho para que você possa acessá-los. Vamos começar abrindo a base `MORADOR` que nos indica cada uma das pessoas recenseadas pela POF e na sequência copiá-la para nossa base principal que trabalharemos ao longo do tutorial. A ideia de manter a base `MORADOR` independente é para que você possa retornar a ela para comparações caso desconfie que algo não tenha funcionado direito.
Aqui utilizamos a biblioteca pyreadr para ler os arquivos `.rds`.


In [ ]:
result = pyreadr.read_r('./Dados/MORADOR.rds')

In [ ]:
MORADOR = result[None]

In [ ]:
base = MORADOR.copy()

Voltaremos mais tarde para a base de moradores a fim de obter o número de integrantes de cada família. Porém, até lá, vamos seguir com os demais questionários para extrair as informações de cada um deles.

### Despesa coletiva

No questionário da **Despesa coletiva** vamos obter o pagamento de dois tipos de impostos, aqueles sobre bens imóveis, incluindo aqui (IPTU, ITR, etc.), além das Contribuições previdenciárias pagas para trabalhadores domésticos.

In [ ]:
DESPESA_COLETIVA = pyreadr.read_r('./Dados/DESPESA_COLETIVA.rds')
DESPESA_COLETIVA = DESPESA_COLETIVA[None]

In [ ]:
DESPESA_COLETIVA = DESPESA_COLETIVA.astype({"V9001": str},errors='ignore')


Abaixo temos uma descrição das variáveis desse questionário que vamos utilizar.
<table>
  <tr>
    <th>Código</th>
    <th>Variável</th>
  </tr>
  <tr>
    <td>V9001</td>
    <td>Produto consumido</td>
  </tr>
  <tr>
    <td>V1904</td>
    <td>Valor da despesa com INSS e outras contribuições</td>
  </tr>
   <tr>
    <td>V8000_DEFLA</td>
    <td>Valor do gasto deflacionado</td>
  </tr>
  <tr>
    <td>V9011</td>
    <td>Nº de meses em que o produto foi adquirido</td>
  </tr>
  <tr>
    <td>FATOR_ANUALIZACAO</td>
    <td>Definido para as variáveis com a V9011 faltante</td>
  </tr>
</table>
Para cada um dos tributos e dos rendimentos vamos criar um objeto que armazene os códigos relacionados a eles, assim é possível modificar apenas essa parte da nossa rotina em eventuais mudanças dos produtos a serem considerados. No caso da `CONTRIBUICAO_PREVIDENCIARIA` criaremos uma lista pois será preciso incluirmos outros códigos para ela, não relacionados ao gasto da família, mas sim com as deduções das rendas do trabalho.

In [ ]:
IMOVEL = ['10006', '10011', '47006', '10007', '10007', '10007', '10012','47007', '10013', '47026', '10014', '10015', '10016', '12035']

In [ ]:
CONTRIBUICAO_PREVIDENCIARIA = {'CONT_PREV_EMP': ['19001','19002','19003','19004','19005','19006','19007','19008','19009','19010','19011','19012','19013','19014','19015','19016','19017','19999']}

Como discutido acima, vamos adaptar cada um dos questionários para que a identificação feita pela `V9001` seja apenas através dos 5 primeiros dígitos.


In [ ]:
DESPESA_COLETIVA.loc[:,'V9001'] = DESPESA_COLETIVA.loc[:,'V9001'].str.slice(0,5)

Outra característica importante da pesquisa é que ela considera os gastos durante todo o ano, sendo necessário mensalizar cada um deles para fazer a comparação com a renda. Para isso, é preciso computar o total dispendido no ano, multiplicando o valor gasto `"V8000_DEFLA"`, pelo número de vezes em que ele foi realizado no ano (`V9011`) e pelo chamado `FATOR_ANUALIZACAO`. Esses dois fatores são complementares entre si, quando um deles possui informações relevantes sobre o gasto, o outro não. Para facilitar o processo de anualização começamos substituindo `V9011` por `1` toda vez que ela for `NA` (faltante).

In [ ]:
DESPESA_COLETIVA['V9011'] = DESPESA_COLETIVA['V9011'].fillna(1)

Na sequência usamos o método `apply` sobre as duas variáveis que nos retornam os valores monetários dos gastos (`"V8000_DEFLA"` e `"V1904_DEFLA"`). Dentro do `apply` utilizamos uma função anônima para mensalizar as despesas.

In [ ]:
DESPESA_COLETIVA.loc[:,["V8000_DEFLA", "V1904_DEFLA"]] = DESPESA_COLETIVA.loc[:,["V8000_DEFLA", "V1904_DEFLA"]].apply(lambda x: x*DESPESA_COLETIVA['V9011']*DESPESA_COLETIVA['FATOR_ANUALIZACAO']/12)

O procedimento acima e os passos a seguir se repetirão durante todo o processo, fazendo as adaptações necessárias para cada um dos questionários, aos poucos se tornará um pouco mais familiar. Agora é preciso criar um indicador capaz de identificar nas linhas da base com que estamos trabalhando os códigos dos produtos que buscamos. Como os tributos sobre a folha de pagamento são indicados por outras variávies que não a `"V8000_DEFLA"`, em alguns questionários criaremos uma lista na qual cada um dos elementos será formado por duas partes. Na primeira, vamos identificar em quais linhas da variável `V9001` temos produtos listados nos grupos que desejamos, algo feito utilizando o método `.isin()`. Enquanto sua segunda parte será responsável por indicar qual variável deve ser utilizada no cômputo de cada um desses grupos de gastos/rendimentos.

Para além da identificação dos impostos, vamos também já computar os chamados rendimentos não monetários, aqueles cuja forma de aquisição descrita pela variável `V9002` registram: `7` - Doação, `8` - Retirada do negócio, `9` - Troca, `10` - Produção própria e `11` - Outra. A metodologia para construção dessa parcela da renda também é descrita nos [*Tradutores de tabelas*](https://www.ibge.gov.br/estatisticas/sociais/saude/24786-pesquisa-de-orcamentos-familiares-2.html?=&t=microdados).

In [ ]:
i_coletiva = {
    'IMOVEL': [DESPESA_COLETIVA['V9001'].isin(IMOVEL),"V8000_DEFLA"],
    'CONT_PREV_EMP': [DESPESA_COLETIVA['V9001'].isin(CONTRIBUICAO_PREVIDENCIARIA['CONT_PREV_EMP']),"V1904_DEFLA"],
    'N_MONET_1':[DESPESA_COLETIVA['V9002'].isin(list(range(7,12))),"V8000_DEFLA"]
}

Por fim, nosso método de agregação para cada família será utilizando a função `groubpy()`, outro *funcional* que nos permite produzir tabelas resumo. Para realizar a agregação nós faremos um ciclo sobre `i_coletiva`, no qual o elemento `indice` será composto pelas duas partes da lista descrita acima na mesma ordem que os criamos. Dessa forma, `indice[0]` será uma sequência de valores lógicos indicando quais linhas devem ser agregadas, enquanto `indice[1]` mostrará qual variável deve ser utilizada para computarmos aquele tributo/rendimento. Com esses dois critérios poderemos agregar para cada uma das UC qual a *soma* dos gastos que desejamos analisar utilizando a função `.sum()`.

A junção da nossa base com as novas informações será feita através da função `merge`, responsável por atribuir as agregações feitas pela função `groupby()` a cada uma das UC. Para isso, devemos indicar quais bases de dados desejamos juntar e qual o critério de conexão entre elas (`on=`). O argumento `how=` indica que queremos manter apenas as observações de uma das bases.

In [ ]:
for indice in i_coletiva.values():
    base = pd.merge(base,
                    DESPESA_COLETIVA[indice[0]].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()[[indice[1]]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

A última etapa para cada um dos questionários, que vamos reproduzir adiante, será nomear as variáveis adicionadas à nossa `base`. Para isso vamos sempre utilizar uma regra para que o tamanho da nossa lista de *indices* (nesse caso o `i_coletiva`) sirva como referência sobre quantas variáveis estamos criando e qual o nome de cada uma delas.

Agora vamos acrescentar a informação sobre o tipo de contratação de serviços domésticos da família (se mensalista, diarista e etc.), fornecida pela variável `V1905` (com o nome *espécie em que o serviço doméstico foi contratado pela UC*) desse mesmo questionário. Para isso, em primeiro lugar vamos retirar todas as respostas *Sem declaração*, alterando o valor `9` para `NaN`. Na sequência vamos criar um vetor lógico para identificar os gastos com trabalhadoras domésticas chamado por `quadro19`. Por fim, a agregação será feita por domicílio, de modo a estabelecer um novo valor possível para a variável `V1905`, imputando a ela o valor `2` para os casos em que a UC conte com mais de uma trabalhadora doméstica. 

In [ ]:
base.set_axis([*base.columns[:base.shape[1]-len(i_coletiva.keys())], *list(i_coletiva.keys())], axis=1, inplace=True)

In [ ]:
DESPESA_COLETIVA.loc[DESPESA_COLETIVA['V1905']==9,'V1905'] = np.NaN

In [ ]:
quadro19 = DESPESA_COLETIVA['V9001'].str.slice(0,2)=='19'

In [ ]:
df_contrempdom = DESPESA_COLETIVA[quadro19].groupby(["COD_UPA","NUM_DOM","NUM_UC"])['V1905'].apply(list).reset_index(name="ContrEmpDom")

In [ ]:
df_contrempdom.loc[:,"ContrEmpDom"] = df_contrempdom.loc[:,"ContrEmpDom"].apply(lambda x: 2 if len(x)>1 else x[0])

In [ ]:
base = pd.merge(base,
                    df_contrempdom, 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

### Rendimentos do trabalho

No questinário de **Rendimentos do trabalho** vamos extrair uma série de informações distintas. Isso porque para além de começarmos a compor o rendimento das famílias, também vamos computar deduções com previdência, imposto de renda e outros descontos.
<table>
  <tr>
    <th>Código</th>
    <th>Variável</th>
  </tr>
  <tr>
    <td>V9001</td>
    <td>Tipo de rendimento</td>
  </tr>
  <tr>
    <td>V531112_DEFLA</td>
    <td>Dedução com previdência pública deflacionada</td>
  </tr>
   <tr>
    <td>V531122_DEFLA</td>
    <td>Dedução com imposto de renda deflacionada</td>
  </tr>
  <tr>
    <td>V531132_DEFLA</td>
    <td>Dedução com ISS e outros impostos deflacionada</td>
  </tr>
  <tr>
    <td>V9010</td>
    <td>Ultimo mes do rendimento</td>
  </tr>
  <tr>
    <td>V9011</td>
    <td>Nº de meses em que o rendimento foi recebido</td>
  </tr>
   <tr>
    <td>FATOR_ANUALIZACAO</td>
    <td>Definido para as variáveis com a V9011 faltante</td>
    </tr>
   <tr>
    <td>V8500_DEFLA</td>
    <td>Valor do rendimento deflacionado</td>
    </tr>
</table>

Contudo, para que possamos reconstituir a `RENDA_TOTAL` reportada na POF, e usada como referência na Nota, faremos uso da tabela de *Tradutor de rendimentos*, também disponível no **Tradutor de tabelas**.

In [ ]:
RENDIMENTO_TRABALHO = pyreadr.read_r('./Dados/RENDIMENTO_TRABALHO.rds')
RENDIMENTO_TRABALHO = RENDIMENTO_TRABALHO[None]

In [ ]:
tradutor_renda = pd.read_excel('./Dados/Tradutor_Rendimento.xlsx')

Como é possível notar, a coluna `Descricao_2` apresenta seis categorias distintas de rendimentos: `"Rendimento do Trabalho"`, `"Transferência"`, `"Rendimento de aluguel"`, `"Outras rendas"` e `"Rendimento Não Monetário"`, para além de uma informação faltante que se refere ao rendimento de variação patrimonial, que veremos mais adiante como construir. Dessas todas, apenas parte da primeira é obtida no questionário de `Rendimento do trabalho`. Para que não tenhamos problemas na identificação dos códigos, vamos excluir a linha referente à variação patrimonial.

In [ ]:
tradutor_renda = tradutor_renda.iloc[:-1]

Dentre os códigos descritos para os rendimentos do trabalho estão alguns que classificaremos em outras categorias, como vale-alimentação (código `54016`), contudo, os únicos códigos incluídos na base *Rendimento do trabalho* são aqueles referentes ao quadro 53 (*Trabablho, rendimentos e deduções no período de referência de 12 meses no trabalho principal ou outro trabalho*). Portanto, quando fizermos a busca da variável `V9001` nos códigos descritos no objeto `TRABALHO` abaixo, não estaremos computando esses outros rendimentos

In [ ]:
tradutor_renda = tradutor_renda.astype({"Codigo": str},errors='ignore')

In [ ]:
TRABALHO = tradutor_renda[tradutor_renda['Descricao_2']=="Rendimento do Trabalho"].loc[:,'Codigo'].str.slice(0,5)

Na sequência vamos descrever os códigos de todas as deduções consideradas pela POF (previdência `"V531112_DEFLA"`, imposto de renda `"V531122_DEFLA"` e outras deduções como ISS `"V531132_DEFLA"`) e depois retomamos os mesmos passos feitos acima.

In [ ]:
DEDUCOES = {'IRPF_TRA': ['53001','53002','53003','53004','53005','53006','48031'], 'DEDUCOES_TRABALHO':['53001','53002','53003','53004','53005','53006'], 'DEDUCOES_NAO_TRABALHO':['54001','54002','54003','54004','54005','54006','54007','54008','54009','54010','54011','54012','54013','54014','54015','54016','54017','54018','54019','54020','54021','54022','54023','54024','54025','54026','54027','54028','54029','54030','54031','54032','54033','54034','54035','55001','55002','55003','55004','55005','55006','55007','55008','55009','55010','55011','55012','55013','55014','55015','55016','55017','55018','55019','55020','55021','55022','55023','55024','55025','55026','55027','55028','55029','55030','55031','55032','55033','55034','55035','55036','55037','55038','55039','55040','55041','55042','55043','55044','55045','55046','55047','55048','55049','55050','55051','55052','55053','55054','55055','55056','55057','55058','55059','55060','55061','55062','55063','55064','55065','55066','55067']}

In [ ]:
CONTRIBUICAO_PREVIDENCIARIA.update({'CONT_PREV_TRABALHO': ['53001','53002','53003','53004','53005','53006']})

In [ ]:
RENDIMENTO_TRABALHO = RENDIMENTO_TRABALHO.astype({"V9001": str},errors='ignore')

In [ ]:
RENDIMENTO_TRABALHO.loc[:,'V9001'] = RENDIMENTO_TRABALHO.loc[:,'V9001'].str.slice(0,5)

In [ ]:
RENDIMENTO_TRABALHO['V9011'] = RENDIMENTO_TRABALHO['V9011'].fillna(1)

In [ ]:
RENDIMENTO_TRABALHO.loc[:,["V531122_DEFLA", "V531112_DEFLA", "V531132_DEFLA", "V8500_DEFLA"]] = RENDIMENTO_TRABALHO.loc[:,["V531122_DEFLA", "V531112_DEFLA", "V531132_DEFLA", "V8500_DEFLA"]].apply(lambda x: x*RENDIMENTO_TRABALHO['V9011']*RENDIMENTO_TRABALHO['FATOR_ANUALIZACAO']/12)

In [ ]:
i_rend_trab = {
    'IRPF_TRA' : [RENDIMENTO_TRABALHO['V9001'].isin(DEDUCOES['IRPF_TRA']),"V531122_DEFLA"],
    'CONT_PREV_TRA' : [RENDIMENTO_TRABALHO['V9001'].isin(CONTRIBUICAO_PREVIDENCIARIA['CONT_PREV_TRABALHO']),"V531112_DEFLA"],
    'DEDUCOES_TRABALHO' : [RENDIMENTO_TRABALHO['V9001'].isin(DEDUCOES['DEDUCOES_TRABALHO']),"V531132_DEFLA"]
}

In [ ]:
for indice in i_rend_trab.values():
    base = pd.merge(base,
                    RENDIMENTO_TRABALHO[indice[0]].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()[[indice[1]]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

In [ ]:
base.set_axis([*base.columns[:base.shape[1]-len(i_rend_trab.keys())], *list(i_rend_trab.keys())], axis=1, inplace=True)

### Construção dos rendimentos - Trabalho

Agora, ao invés de identificarmos os rendimentos monetários do trabalho para a UC, vamos identificar para cada um dos indivíduos, utilizando ainda a função `aggregate`, mas adicionando a variável `"COD_INFORMANTE"` na informação do agrupamento. Vamos atribuir o nome da variável como `"TRABALHO_1"` pois ainda teremos outro rendimento do trabalho mais adiante.

In [ ]:
base = pd.merge(base,
                    RENDIMENTO_TRABALHO[RENDIMENTO_TRABALHO['V9001'].isin(TRABALHO)].groupby(["COD_UPA","NUM_DOM","NUM_UC","COD_INFORMANTE"]).sum()[["V8500_DEFLA"]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC", "COD_INFORMANTE"])

In [ ]:
base.set_axis([*base.columns[:-1], 'TRABALHO_1'], axis=1, inplace=True)

### Outros rendimentos

No questionário de *Outros rendimentos* faremos duas etapas. A primeira consiste nos passos que já temos feito, enquanto a segunda será o cálculo de uma parte da chamada *variação patrimonial*, para a qual seguiremos as instruções dadas nos *Tradutores de tabelas* que discutimos acima.


In [ ]:
OUTROS_RENDIMENTOS = pyreadr.read_r('./Dados/OUTROS_RENDIMENTOS.rds')
OUTROS_RENDIMENTOS = OUTROS_RENDIMENTOS[None]

<table>
  <tr>
    <th>Código</th>
    <th>Variável</th>
  </tr>
  <tr>
    <td>V9001</td>
    <td>Tipo de rendimento</td>
  </tr>
  <tr>
    <td>V8500_DEFLA</td>
    <td>Valor do rendimento deflacionado</td>
  </tr>
   <tr>
    <td>V8501_DEFLA</td>
    <td>Valor da dedução deflacionado</td>
  </tr>
  <tr>
    <td>V9010</td>
    <td>Ultimo mes do rendimento</td>
  </tr>
  <tr>
    <td>V9011</td>
    <td>Nº de meses em que o rendimento foi recebido</td>
  </tr>
   <tr>
    <td>FATOR_ANUALIZACAO</td>
    <td>Definido para as variáveis com a V9011 faltante</td>
    </tr>
</table>

Todos os códigos indicados a seguir são de transfêrencias governamentais, algumas do sistema de previdência, como as aposentadorias do Regime Geral da Previdência Social e do Regime Próprio de Previdência Social (exclusiva do funcionalismo público), outras de caráter assistencial, como o Programa Bolsa Família e o Benefício de Prestação Continuada.

In [ ]:
RGPS = ['54004','54005','55003','55004','55050']

RPPS = ['54006','54007','55005','55006']

AUXILIO_PUBLICO = ['54009','54024','54026','54027','54030','54034','55036','55045','55056','55062','55065','55066']

AUXILIO_MISTO = ['54016','54017','54020','54022','54023','54025','54032','54033','55047','55055']

BOLSA_FAMILIA = ['54001','54003','55018']

BPC = ['54002']

BOLSA_ESTUDO = ['54010']

SEGURO_DESEMPREGO = ['55017']

Para além dessas, vamos identificar outros rendimentos classificados como transferências através dos *Tradutores de rendimentos*, além dos rendimentos de aluguel e outras rendas. Para não fazermos dupla contagem de nenhuma das transferências vamos selecionar os códigos indicados no *tradutor de rendimentos* excluindo aquelas já identificadas, para as quais também nos baseamos em Gaiger, 2020.


In [ ]:
TRANSFERENCIAS = list(set(tradutor_renda[tradutor_renda['Descricao_2']=='Transferência']['Codigo'].str.slice(0,5))-set([*RGPS,*RPPS,*AUXILIO_PUBLICO,*AUXILIO_MISTO,*BOLSA_FAMILIA, *BPC,*BOLSA_ESTUDO,*SEGURO_DESEMPREGO]))

In [ ]:
ALUGUEL = tradutor_renda[tradutor_renda['Descricao_2']=="Rendimento de aluguel"].loc[:,'Codigo'].str.slice(0,5)

In [ ]:
OUTRAS_RENDAS = tradutor_renda[tradutor_renda['Descricao_2']=="Outras rendas"].loc[:,'Codigo'].str.slice(0,5)

Nessa parte retornamos aos mesmos passos anteriores.

In [ ]:
OUTROS_RENDIMENTOS = OUTROS_RENDIMENTOS.astype({"V9001": str},errors='ignore')

In [ ]:
OUTROS_RENDIMENTOS.loc[:,'V9001'] = OUTROS_RENDIMENTOS.loc[:,'V9001'].str.slice(0,5)

In [ ]:
OUTROS_RENDIMENTOS['V9011'] = OUTROS_RENDIMENTOS['V9011'].fillna(1)

In [ ]:
OUTROS_RENDIMENTOS.loc[:,["V8500_DEFLA", "V8501_DEFLA"]] = OUTROS_RENDIMENTOS.loc[:,["V8500_DEFLA", "V8501_DEFLA"]].apply(lambda x: x*OUTROS_RENDIMENTOS['V9011']*OUTROS_RENDIMENTOS['FATOR_ANUALIZACAO']/12)

Agora, como vamos trabalhar apenas com rendimentos, nossa lista dos índices será utilizada apenas na agregação por indivíduo e todos na mesma variável (`V8500_DEFLA`).

In [ ]:
i_out_rend = {
    'RGPS_T': OUTROS_RENDIMENTOS['V9001'].isin(RGPS),
    'RPPS_T': OUTROS_RENDIMENTOS['V9001'].isin(RPPS),
    'AUXI_PUB_T': OUTROS_RENDIMENTOS['V9001'].isin(AUXILIO_PUBLICO),
    'AUX_MIS_T': OUTROS_RENDIMENTOS['V9001'].isin(AUXILIO_MISTO),
    'PBF_T': OUTROS_RENDIMENTOS['V9001'].isin(BOLSA_FAMILIA),
    'BPC_T': OUTROS_RENDIMENTOS['V9001'].isin(BPC),
    'BOL_EST_T': OUTROS_RENDIMENTOS['V9001'].isin(BOLSA_ESTUDO),
    'SEG_DES_T': OUTROS_RENDIMENTOS['V9001'].isin(SEGURO_DESEMPREGO),
    'ALUGUEL': OUTROS_RENDIMENTOS['V9001'].isin(ALUGUEL),
    'OUTRAS_RENDAS': OUTROS_RENDIMENTOS['V9001'].isin(OUTRAS_RENDAS),
    'TRABALHO_2': OUTROS_RENDIMENTOS['V9001'].isin(set(TRABALHO)-set([*AUXILIO_MISTO,*AUXILIO_PUBLICO])),
    'OUTRAS_TRANSFERENCIAS_T' : OUTROS_RENDIMENTOS['V9001'].isin(TRANSFERENCIAS)
}

In [ ]:
for indice in i_out_rend.values():
    base = pd.merge(base,
                    OUTROS_RENDIMENTOS[indice].groupby(["COD_UPA","NUM_DOM","NUM_UC","COD_INFORMANTE"]).sum()[['V8500_DEFLA']], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC","COD_INFORMANTE"])

In [ ]:
base.set_axis([*base.columns[:base.shape[1]-len(i_out_rend.keys())], *list(i_out_rend.keys())], axis=1, inplace=True)

Agora será preciso computarmos as *Deduções não trabalho* conforme indicadas em Gaiger, 2020. Para elas faremos a agregação no nível da UC, como temos feito com os tributos.

In [ ]:
DEDUCOES_N_T = OUTROS_RENDIMENTOS['V9001'].isin(DEDUCOES['DEDUCOES_NAO_TRABALHO'])

In [ ]:
base = pd.merge(base,
                    OUTROS_RENDIMENTOS[DEDUCOES_N_T].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()[['V8501_DEFLA']], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

In [ ]:
base.set_axis([*base.columns[:-1], 'DEDUCOES_N_T'], axis=1, inplace=True)

### Variação patrimonial

Para a construção dessa variável não seguiremos exatamente o procedimento descrito pelo *Tradutor de tabelas*. Ao invés de computarmos a primeira parte da variável para a UC, faremos por indivíduo, dado que assim temos um ajuste final melhor da variável com as informações divulgadas pelo IBGE.

In [ ]:
VP1 = ['55008', '55010', '55016', '55020', '55021', '55022', '55023', '55024', '55025', '55026', '55035', '55037', '55044', '55053', '55061']

In [ ]:
base = pd.merge(base,
                    OUTROS_RENDIMENTOS[OUTROS_RENDIMENTOS['V9001'].isin(VP1)].groupby(["COD_UPA","NUM_DOM","NUM_UC","COD_INFORMANTE"]).sum()[["V8500_DEFLA"]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC", "COD_INFORMANTE"])

In [ ]:
base.set_axis([*base.columns[:-1], 'VP1'], axis=1, inplace=True)

Agora faremos o cálculo das diferenças indicadas na parte 2 da construção da variação patrimonial, seguindo as operações explicadas no *tradutor de tabelas*.

In [ ]:
VP2 = ['57001', '56001', '57002', '56002', '57003', '56003', '57004', '56004']

In [ ]:
for produto in VP2:
    base = pd.merge(base,
                    OUTROS_RENDIMENTOS[OUTROS_RENDIMENTOS['V9001'] == produto].groupby(["COD_UPA","NUM_DOM","NUM_UC","COD_INFORMANTE"]).sum()[["V8500_DEFLA"]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC", "COD_INFORMANTE"])
    base.set_axis([*base.columns[:-1], produto], axis=1, inplace=True)

In [ ]:
base.fillna(0, inplace = True)

base['VP.1'] = base['57001'] - base['56001']
base.loc[base['VP.1']<0, 'VP.1'] = 0
base['VP.2'] = base['57002'] - base['56002']
base.loc[base['VP.2']<0, 'VP.2'] = 0
base['VP.3'] = base['57003'] - base['56003']
base.loc[base['VP.3']<0, 'VP.3'] = 0
base['VP.4'] = base['57004'] - base['56004']
base.loc[base['VP.4']<0, 'VP.4'] = 0
base['VP2'] = base['VP.1'] + base['VP.2'] + base['VP.3'] + base['VP.4']

base.drop(base.columns[list(range(79,91))],axis=1, inplace = True)

Em seguida vamos agregar as informações de cada informante para a UC.

In [ ]:
base['VP_INFOR'] = base['VP1']+base['VP2']
base = pd.merge(base,
                    base.groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()[['VP_INFOR']], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
base.set_axis([*base.columns[:-2], 'VP_INFOR', 'VP'], axis=1, inplace=True)

### Despesa individual

Para esse questionário seguiremos os passos anteriores considerando pagamento de tributos e os rendimentos não monetários.

In [ ]:
DESPESA_INDIVIDUAL = pyreadr.read_r('./Dados/DESPESA_INDIVIDUAL.rds')
DESPESA_INDIVIDUAL = DESPESA_INDIVIDUAL[None]

<table>
  <tr>
    <th>Código</th>
    <th>Variável</th>
  </tr>
  <tr>
    <td>V9001</td>
    <td>Tipo de rendimento</td>
  </tr>
   <tr>
    <td>V9002</td>
    <td>Forma de aquisição</td>
  </tr>
  <tr>
    <td>V8000_DEFLA</td>
    <td>Valor deflacionado do gasto</td>
  </tr>
  <tr>
    <td>V9011</td>
    <td>Nº de meses em que o rendimento foi recebido</td>
  </tr>
   <tr>
    <td>FATOR_ANUALIZACAO</td>
    <td>Definido para as variáveis com a V9011 faltante</td>
    </tr>
</table>
Notar que foi incluído `IRPF2` para cobrir o código `48031`, referente à complementação do IR e impostos referentes ao exercício anterior.


In [ ]:
CONTRIBUICAO_PREVIDENCIARIA.update({'CONT_PREV_INDIVIDUAL':['48005','48037']})

IPVA = ['50001','50002','50004','50005','50006','50007','50008','50014','50016','50017']

OUTROS_DIRETOS = ['48004','48010','48011']

In [ ]:
DESPESA_INDIVIDUAL = DESPESA_INDIVIDUAL.astype({"V9001": str},errors='ignore')
DESPESA_INDIVIDUAL.loc[:,'V9001'] = DESPESA_INDIVIDUAL.loc[:,'V9001'].str.slice(0,5)
DESPESA_INDIVIDUAL['V9011'].fillna(1, inplace = True)

Como esse questionário utiliza apenas o `FATOR_ANUALIZACAO`, sem a variável `V9011`, então podemos passar direto para a mensalização do gasto.

In [ ]:
DESPESA_INDIVIDUAL.loc[:,"V8000_DEFLA"] = DESPESA_INDIVIDUAL["V8000_DEFLA"]*DESPESA_INDIVIDUAL['V9011']*DESPESA_INDIVIDUAL['FATOR_ANUALIZACAO']/12

Por fim, dado que essa será a única informação relevante desse questionário, faremos sua agregação apenas indicando a forma de aquição do bem, através da variável `V9002`.

In [ ]:
i_individual = {
    'CONT_PREV_INDIVIDUAL': DESPESA_INDIVIDUAL['V9001'].isin(CONTRIBUICAO_PREVIDENCIARIA['CONT_PREV_INDIVIDUAL']),
    'IPVA': DESPESA_INDIVIDUAL['V9001'].isin(IPVA),
    'OUT_DIR': DESPESA_INDIVIDUAL['V9001'].isin(OUTROS_DIRETOS),
    'IRPF_IND': DESPESA_INDIVIDUAL['V9001'].isin(DEDUCOES['IRPF_TRA']),
    'IMOVEL_OUTROS': DESPESA_INDIVIDUAL['V9001'].isin(IMOVEL),
    'N_MONET_2': DESPESA_INDIVIDUAL['V9002'].isin(list(range(7,12)))
}

In [ ]:
for indice in i_individual.values():
    base = pd.merge(base,
                    DESPESA_INDIVIDUAL[indice].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['V8000_DEFLA'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

In [ ]:
base.set_axis([*base.columns[:base.shape[1]-len(i_individual.keys())], *list(i_individual.keys())], axis=1, inplace=True)

### Caderneta coletiva

No caso desse questinário, responsável exclusivamente pela mensuração dos gastos alimentares, vamos recuperar apenas informações da renda não monetária.

In [ ]:
CADERNETA_COLETIVA = pyreadr.read_r('./Dados/CADERNETA_COLETIVA.rds')
CADERNETA_COLETIVA = CADERNETA_COLETIVA[None]

<table>
  <tr>
    <th>Código</th>
    <th>Variável</th>
  </tr>
  <tr>
    <td>V9001</td>
    <td>Tipo de rendimento</td>
  </tr>
   <tr>
    <td>V9002</td>
    <td>Forma de aquisição</td>
  </tr>
  <tr>
    <td>V8000_DEFLA</td>
    <td>Valor da aquisição</td>
  </tr>
  <tr>
    <td>V9011</td>
    <td>Nº de meses em que o produto foi adquirido</td>
  </tr>
   <tr>
    <td>FATOR_ANUALIZACAO</td>
    <td>Definido para as variáveis com a V9011 faltante</td>
    </tr>
</table>
Nessa base não será preciso criarmos a lista dos gastos de interesse, faremos apenas os ajustes das variáveis e poderemos computar a soma desses recebimentos.

In [ ]:
CADERNETA_COLETIVA = CADERNETA_COLETIVA.astype({"V9001": str},errors='ignore')
CADERNETA_COLETIVA.loc[:,'V9001'] = CADERNETA_COLETIVA.loc[:,'V9001'].str.slice(0,5)

Como esse questionário utiliza apenas o `FATOR_ANUALIZACAO`, sem a variável `V9011`, então podemos passar direto para a mensalização do gasto.

In [ ]:
CADERNETA_COLETIVA.loc[:,"V8000_DEFLA"] = CADERNETA_COLETIVA["V8000_DEFLA"]*CADERNETA_COLETIVA['FATOR_ANUALIZACAO']/12

Por fim, dado que essa será a única informação relevante desse questionário, faremos sua agregação apenas indicando a forma de aquição do bem, através da variável `V9002`.

In [ ]:
base = pd.merge(base,
                    CADERNETA_COLETIVA[CADERNETA_COLETIVA['V9002'].isin(list(range(7,12)))].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['V8000_DEFLA'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

In [ ]:
base.set_axis([*base.columns[:-1], 'N_MONET_3'], axis=1, inplace=True)

### Aluguel estimado

Esse questionário consiste na segunda parte do cálculo da renda não monetária conforme descrita no **Tradutor de rendimentos**. Ele é responsável por atribuir um valor de aluguel para as UC residentes em casa própria, ou sem que precisem arcar com o aluguel. Aqui é preciso identificarmos o total do *Alguel estimado* e subtrair os *serviços para domicílio* pagos em dinheiro pela UC.

In [ ]:
ALUGUEL_ESTIMADO = pyreadr.read_r('./Dados/ALUGUEL_ESTIMADO.rds')
ALUGUEL_ESTIMADO = ALUGUEL_ESTIMADO[None]
ALUGUEL_ESTIMADO['V9011'].fillna(1, inplace = True)

<table>
  <tr>
    <th>Código</th>
    <th>Variável</th>
  </tr>
  <tr>
    <td>V8000_DEFLA</td>
    <td>Valor da aquisição</td>
  </tr>
  <tr>
    <td>V9011</td>
    <td>Nº de meses em que o produto foi adquirido</td>
  </tr>
   <tr>
    <td>FATOR_ANUALIZACAO</td>
    <td>Definido para as variáveis com a V9011 faltante</td>
    </tr>
</table>
Nesse questionário, apesar das variáveis `V9001` e `V9002` estarem presentes, cada uma apresenta um único valor. A primeira indica o código de rendimento `000101` (aluguel estimado), enquanto a segunda indica que a forma de aquisição foi `11` - Outra. Assim, novamente o processo de agregação será mais simples, apenas necessitando realizar a soma para cada UC.

In [ ]:
ALUGUEL_ESTIMADO.loc[:,"V8000_DEFLA"] = ALUGUEL_ESTIMADO["V8000_DEFLA"]*ALUGUEL_ESTIMADO["V9011"]*ALUGUEL_ESTIMADO['FATOR_ANUALIZACAO']/12

In [ ]:
base = pd.merge(base,
                    ALUGUEL_ESTIMADO.groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['V8000_DEFLA'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
base.set_axis([*base.columns[:-1], 'ALUGUEL_EST_1'], axis=1, inplace=True)

Na sequência será preciso retirar os gastos monetários do questionário de *Consumo coletivo* associados ao imóvel: *Quadro 8* - conservação, manutenção e pequenos reparos; *Quadro 10* - alugel, impostos e outras taxas; e *Quadro 12* - outros itens do domicílio. Teremos dois índices nesse caso, o primeiro para identificar esses gastos e o segundo para identificar quando a forma de aquisição envolveu gastos monetários.

In [ ]:
i_ALUGUEL_EST_1 = DESPESA_COLETIVA['V9001'].isin(list(map(str,[*list(range(80010,80242)),*list(range(80260,80681)),89990,10006,10011,*list(range(12005,12009)),*list(range(12010,12016)),*list(range(12017,12021)),*list(range(12023,12026)),*list(range(12027,12037)),12999])))
i_ALUGUEL_EST_2 = DESPESA_COLETIVA['V9002'].isin(list(range(1,7)))

In [ ]:
base = pd.merge(base,
                    DESPESA_COLETIVA[i_ALUGUEL_EST_1 & i_ALUGUEL_EST_2].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['V8000_DEFLA'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

In [ ]:
base.set_axis([*base.columns[:-1], 'ALUGUEL_EST_2'], axis=1, inplace=True)

Agora vamos consolidar as informações do aluguel estimado que agregamos acima.

In [ ]:
base.fillna(0, inplace = True)
base['ALUGUEL_EST'] = base['ALUGUEL_EST_1'] - base['ALUGUEL_EST_2']
base.loc[base['ALUGUEL_EST']<0, 'ALUGUEL_EST'] = 0

### Serviços não monetários

Os últimos questionários que vamos utilizar referem-se aos serviços não monetários, uma novidade da POF 2017-2018. Esses questionários destinam-se à compreensão do recebimento de serviços não monetários de educação e saúde. Faremos a agregação das informações de cada um dos questionários para composição da renda não monetária.

In [ ]:
SERVICO_NAO_MONETARIO_POF2 = pyreadr.read_r('./Dados/SERVICO_NAO_MONETARIO_POF2.rds')
SERVICO_NAO_MONETARIO_POF2 = SERVICO_NAO_MONETARIO_POF2[None]
SERVICO_NAO_MONETARIO_POF2['V9011'].fillna(1, inplace = True)

In [ ]:
SERVICO_NAO_MONETARIO_POF2.loc[:,"V8000_DEFLA"] = SERVICO_NAO_MONETARIO_POF2["V8000_DEFLA"]*SERVICO_NAO_MONETARIO_POF2["V9011"]*SERVICO_NAO_MONETARIO_POF2['FATOR_ANUALIZACAO']/12

In [ ]:
base = pd.merge(base,
                    SERVICO_NAO_MONETARIO_POF2.groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['V8000_DEFLA'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
base.set_axis([*base.columns[:-1], 'N_MONET_4'], axis=1, inplace=True)

In [ ]:
SERVICO_NAO_MONETARIO_POF4 = pyreadr.read_r('./Dados/SERVICO_NAO_MONETARIO_POF4.rds')
SERVICO_NAO_MONETARIO_POF4 = SERVICO_NAO_MONETARIO_POF4[None]
SERVICO_NAO_MONETARIO_POF4['V9011'].fillna(1, inplace = True)

In [ ]:
SERVICO_NAO_MONETARIO_POF4.loc[:,"V8000_DEFLA"] = SERVICO_NAO_MONETARIO_POF4["V8000_DEFLA"]*SERVICO_NAO_MONETARIO_POF4["V9011"]*SERVICO_NAO_MONETARIO_POF4['FATOR_ANUALIZACAO']/12

In [ ]:
base = pd.merge(base,
                    SERVICO_NAO_MONETARIO_POF4.groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['V8000_DEFLA'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
base.set_axis([*base.columns[:-1], 'N_MONET_5'], axis=1, inplace=True)

### Ajustes finais

#### Durezas da pesquisa

O primeiro dos ajustes finais é daqueles típicos dos problemas de pesquisa que só é possível descobrir fazendo. Trata-se do fato de uma das famílias aparecer com renda zero, mas uma cobrança de impostos sobre imóveis não nula, fazendo com que sua participação na amostra distorça os resultados.

In [ ]:
base.loc[(base['COD_UPA']==330102475) & (base['NUM_DOM']==8) & (base['NUM_UC']==2),'IMOVEL'] = 0
base.loc[(base['COD_UPA']==330102475) & (base['NUM_DOM']==8) & (base['NUM_UC']==2),'IMOVEL_OUTROS'] = 0

### Composição da Unidade de Consumo

Agora vamos obter número de integrantes de cada *Unidade de Consumo*. Repare que para isso utilizaremos o tamanho do vetor formado pela variável *código do informante* de cada *UC*. Antes, será preciso retirarmos da amostra os integrantes da UC que são empregados domésticos ou parentes de empregados domésticos, identificados pelos valores `18:19` pela variável `V0306`, que indica a `Condição na Unidade de Consumo`.

In [ ]:
base.loc[base['V0306'].isin([18,19]),'COD_INFORMANTE'] = np.NaN

In [ ]:
base = pd.merge(base,
                    base[pd.notna(base['COD_INFORMANTE'])].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).count()['COD_INFORMANTE'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
base.rename(columns={'COD_INFORMANTE_x':'COD_INFORMANTE'}, inplace = True)
base.set_axis([*base.columns[:-1], 'N_Pessoas'], axis=1, inplace=True)

Agora vamos adicionar a soma do peso das observações de cada indivíduo que compõem as UC. Com isso teremos uma variável indicando quantas pessoas aquela unidade de consumo representa conjuntamente, excluindo novamente empregados domésticos e seus parentes

In [ ]:
base = pd.merge(base,
                    base[pd.notna(base['COD_INFORMANTE'])].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum()['PESO_FINAL'], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
base.set_axis([*base.columns[:-1], 'PESO_UC'], axis=1, inplace=True)

### Renda não monetária

A partir do tamanho das famílias, vamos construir a renda não monetária per capita da família.

In [ ]:
base.fillna(0, inplace=True)

In [ ]:
base['RENDA_N_M'] = base['N_MONET_1']+base['N_MONET_2']+base['N_MONET_3']+base['ALUGUEL_EST']+base['N_MONET_4']+base['N_MONET_5']
base[base['V0306'].isin([18,19])] = 0
base['RENDA_N_M_PC'] = base['RENDA_N_M']/base['N_Pessoas']

### Renda monetária

Agora vamos agregar a renda monetária de todos os informantes da UC, a fim de criarmos uma variável para a renda monetária per capita. Faremos isso em duas etapas, na primeira agregaremos o total de cada um dos rendimentos dentro da família, na sequência vamos somar todos esses rendimentos na variável `RENDA_MONET_UC` e fazer a junção com a nossa base.

In [ ]:
RENDA_MONET_UC = pd.DataFrame(base[~(base['V0306'].isin([18,19]))].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum().iloc[:,list(range(61,74))])
RENDA_MONET_UC.reset_index(inplace=True)

In [ ]:
RENDA_MONET_UC['RENDA_MONET_UC'] = RENDA_MONET_UC.drop(RENDA_MONET_UC.columns[list(range(0,3))],axis=1).sum(axis=1)

In [ ]:
base = pd.merge(base,
                    RENDA_MONET_UC[["COD_UPA","NUM_DOM","NUM_UC","RENDA_MONET_UC"]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])

In [ ]:
base[base['V0306'].isin([18,19])]['RENDA_MONET_UC'] = 0

In [ ]:
base['RENDA_M_PC'] = base['RENDA_MONET_UC']/base['N_Pessoas']

Na sequência faremos algo similar, mas ao invés de somarmos a renda de toda a UC, vamos considerar apenas a renda do informante.

In [ ]:
base['RENDA_M_INFOR'] = base.iloc[:,list(range(64,77))].sum(axis=1)

### Ajuste de variáveis

Agora vamos unificar variáveis, para manter apenas as informações mais relevantes de cada uma delas.

In [ ]:
base['TRABALHO'] = base['TRABALHO_1'] + base['TRABALHO_2']
base.drop(['TRABALHO_1','TRABALHO_2'], axis = 1, inplace=True)

base['IRPF'] = base['IRPF_TRA']+base['IRPF_IND']
base.drop(['IRPF_TRA','IRPF_IND'], axis = 1, inplace=True)

base['CONTR_PREVI'] = base['CONT_PREV_INDIVIDUAL']+base['CONT_PREV_TRA']
base.drop(['CONT_PREV_INDIVIDUAL','CONT_PREV_TRA'], axis = 1, inplace=True)

base['IMOVEL'] = base['IMOVEL']+base['IMOVEL_OUTROS']
base.drop(['IMOVEL_OUTROS'], axis = 1, inplace=True)

In [ ]:
base = base.iloc[:,[*list(range(0,59)),*list(range(60,74)),77,*list(range(78,80)),84,*list(range(87,len(base.columns)))]] 

### Per capitalizar tributos e rendimentos

Na sequência dividimos todos os impostos e os rendimentos pelo número de moradores da casa e, por fim, anexamos todas as nossas informações na base de indivíduos. Repare que a per capitalização é feita dentro do ciclo, no qual selecionamos a coluna da variável desejada e dividimos pelo número de pessoas da UC.

In [ ]:
tributos = ["CONT_PREV_EMP", 
              "DEDUCOES_TRABALHO", 
              "DEDUCOES_N_T", 
              "CONTR_PREVI", 
              "IPVA", 
              "OUT_DIR", 
              "IRPF", 
              "IMOVEL"]

for t in tributos:
    base['tributo.pc'] = base[t]/base['N_Pessoas']
    base.set_axis([*base.columns[:-1], str(t+'_PC')], axis=1, inplace=True)

In [ ]:
rendimentos = ["RGPS_T",
                 "RPPS_T",
                 "AUXI_PUB_T",
                 "AUX_MIS_T",
                 "PBF_T",
                 "BPC_T",
                 "BOL_EST_T",
                 "SEG_DES_T",
                 "ALUGUEL",
                 "OUTRAS_RENDAS",
                 "OUTRAS_TRANSFERENCIAS_T",
                 "TRABALHO",
                 "VP"]
for r in rendimentos:
    aux_rendimentos=pd.merge(base[["COD_UPA","NUM_DOM","NUM_UC","N_Pessoas"]],
                    pd.DataFrame(base[~(base['V0306'].isin([18,19]))].groupby(["COD_UPA","NUM_DOM","NUM_UC"]).sum().loc[:,r]), 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC"])
    base['rendimento.pc'] = aux_rendimentos[r]/aux_rendimentos['N_Pessoas']
    base.set_axis([*base.columns[:-1], str(r+'_PC')], axis=1, inplace=True)

Por fim, para além de todas as informações já coletadas, também vamos coletar no questionário de Rendimentos do Trabalho as informações a respeito da chamada `posição na ocupação`, ou seja, se a pessoa era empregada do setor público ou privado, empregadora, trabalhadora doméstica, militar, conta própria ou tinha um trabalho não remunerado junto a outro parente. Essas informações são fornecidas pela variável `V5302`. Note que é preciso recortar a base `RENDIMENTO_TRABALHO` para que ela reporte apenas as informações referentes ao trabalho principal, através da variável `SUB_QUADRO`.


In [ ]:
base = pd.merge(base,
                    RENDIMENTO_TRABALHO[RENDIMENTO_TRABALHO['SUB_QUADRO']==1][["COD_UPA","NUM_DOM","NUM_UC","COD_INFORMANTE", "V5302"]], 
                    how = 'left',
                    on=["COD_UPA","NUM_DOM","NUM_UC", "COD_INFORMANTE"])
base.set_axis([*base.columns[:-1], 'PosiOcup'], axis=1, inplace=True)
base['PosiOcup'].fillna(0,inplace=True)

In [ ]:
base.rename(columns={'PESO_FINAL_x':'PESO_FINAL'},inplace=True)

### Salvar

Finalmente.

In [ ]:
base.to_csv('POF_Teste_v1.csv',index=False)

# Exercícios

**1)** De que maneira poderíamos adaptar o *ciclo* de geração de variáveis, por exemplo para o qustionário de *Despesas Coletivas*, para que o nome delas não precisasse ser estabelecido fora dele?


**2)** Note a diferença entre os `# Índices` construídos para o questionário de *Despesas coletivas* e *Outros rendimentos*. De que maneira isso altera o uso do índice dentro do ciclo?

**3)** Vamos trabalhar com a conferência das rendas *monetária* e *não monetária* geradas ao longo do tutorial com aquelas reportadas pelo IBGE.

a.  Faça a divisão da soma dos rendimentos da nossa base pela soma dos rendimentos indicados na base `MORADOR`, considerando o peso de cada indivíduo.

**4)** Faça agora a comparação da `RENDA_TOTAL` com os componentes dela que temos (renda monetária, não monetária e variação patrimonial). Note que ela está na base `MORADOR` ainda somada para toda a UC, sendo necessário dividí-la pela nossa variável de número de pessoas na UC. Você consegue pensar uma forma de reduzir a diferença entre elas?

